In [ ]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

In [ ]:
#dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
     #  'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
      # 'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       #'Target':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}


In [ ]:
#df = pd.DataFrame(dataset,columns=['Taste','Temperature','Texture','Target'])

df = pd.read_csv('/content/telco_preprocessed.csv', error_bad_lines=False)
#df = df.drop(['customerID'], axis=1)

In [ ]:
df.head(100)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Target
0,Female,0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,4.0,0.0,0
1,Male,0,No,No,11.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,7.0,11.0,0
2,Male,0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,7.0,2.0,1
3,Male,0,No,No,13.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),5.0,11.0,0
4,Female,0,No,No,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,10.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Female,0,No,No,6.0,Yes,Yes,Fiber optic,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,11.0,7.0,1
96,Male,0,Yes,Yes,18.0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,One year,Yes,Credit card (automatic),9.0,16.0,0
97,Male,0,No,No,3.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,3.0,2.0,1
98,Male,0,No,No,14.0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),3.0,8.0,0


In [ ]:
def root_entropy(df):
  entropy = 0
  target_values = df[df.keys()[-1]].unique()
  total_count = len(df)
  for value in target_values:
    #print(value)
    value_count = len(df['Target'][df['Target'] == value])
    #print(value_count)
    p_i = value_count/(total_count + eps)
    entropy += -(p_i * log(p_i + eps))
  #print("Entropy at the root is: " + str(entropy))
  return entropy
  
root_entropy(df)

0.8347419473972106

In [ ]:
def attribute_entropy(df, attribute):
  attribute_values = df[attribute].unique()
  target_values = df['Target'].unique()

  #print(attribute_values)
  total_count = len(df)
  E_attribute = 0
  for value in attribute_values:
    value_count = len(df[attribute][df[attribute] == value])
    frac1 = -value_count/(total_count + eps)
    E_value = 0
    for target_value in target_values:
      target_value_count = len(df[attribute][df[attribute] == value][df['Target'] == target_value])
      p_i = (target_value_count/(value_count + eps))
      E_value += p_i * log(p_i + eps)
    E_value = frac1 * E_value
    #print("Entropy for value - " + str(value) + " is: " + str(E_value))
    E_attribute += E_value
  #print("Entropy of attribute - " + str(attribute) + " is: " + str(E_attribute))
  return abs(E_attribute)


In [ ]:
def IG (df, attribute):
  E_0 = root_entropy(df)
  E_attribute = attribute_entropy(df, attribute)
  ig = E_0 - E_attribute
  #print("Information gain for attribute - " + str(attribute) + " is: " + str(ig))
  return ig


In [ ]:
def Importance(attributes, df):
  max_ig = -999999
  selected_attr = ""
  for attribute in attributes:
    ig_attr = IG(df, attribute)
    if ig_attr > max_ig:
      selected_attr = attribute
      max_ig = ig_attr
  return selected_attr

In [ ]:
def get_exs(df, attribute, value):
  return df[df[attribute] == value].reset_index(drop=True)

import statistics
from statistics import mode

def plurality_value(df):
  targets = list(df['Target'])
  counter = 0
  mode = targets[0] 
  for i in targets: 
    curr_frequency = targets.count(i) 
    if(curr_frequency > counter): 
        counter = curr_frequency 
        mode = i
  return mode 


def Decision_Tree_Learning(examples, attributes, depth, max_depth, flag, tree=None):
  depth += 1
  a = [] #a is all attributes that we recived. Please work further with a and not attributes
  for attribute in attributes:
     a.append(attribute)
     ###########################################
  A = Importance(a, examples) # A is the selected node, that is the one with the most IG
  attribute_values = np.unique(df[A])
  #print(attribute_values)
  if tree is None:                    
    tree={}
    tree[A] = {}

  #This portion is for finding the remaining attributes
  #attributes.remove(A)
  #remaining_attributes = attributes
  #attributes = a

  remaining_attributes = []
  for attribute in a:
    if attribute != A:
      remaining_attributes.append(attribute)
  #print(remaining_attributes)

  for value in attribute_values:
    exs = get_exs(examples, A, value)
    #print (value, exs)

    #depth checking here:
    if flag == False: ##Let the tree grow as much as it can
      #condition checking here:
      if len(exs) == 0:
        tree[A][value] = plurality_value(examples)
      elif len(remaining_attributes) == 0:
        tree[A][value] = plurality_value(exs)
      else:
        target_values, target_values_count = np.unique(exs['Target'],return_counts=True)
        if len(target_values_count) == 1: #if all examples have the same classification, return the classification
          tree[A][value] = target_values[0]
        else: ### We have rows and unchecked columns
          tree[A][value] = Decision_Tree_Learning(exs, remaining_attributes, depth, max_depth, False)
    else: #check depth
      if depth == max_depth:
        if len(exs) == 0:
          tree[A][value] = plurality_value(examples)
        else:
          tree[A][value] = plurality_value(exs)
      elif depth < max_depth:
        if len(exs) == 0:
          tree[A][value] = plurality_value(examples)
        elif len(remaining_attributes) == 0:
          tree[A][value] = plurality_value(exs)
        else:
          target_values, target_values_count = np.unique(exs['Target'],return_counts=True)
          if len(target_values_count) == 1: #if all examples have the same classification, return the classification
            tree[A][value] = target_values[0]
          else: ### We have rows and unchecked columns
            tree[A][value] = Decision_Tree_Learning(exs, remaining_attributes, depth, max_depth, True)

  return tree


In [ ]:
all_columns = list(df)
y_column = all_columns.pop(all_columns.index('Target'))
x_columns = all_columns

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

max_depth = 1
tree = Decision_Tree_Learning(train, x_columns, -1, max_depth-1, False)

import pprint
pprint.pprint(tree)

Streaming output truncated to the last 5000 lines.
                                                                                                                                                                                                                                                                                         'Yes': {'MultipleLines': {'No': {'OnlineSecurity': {'No': {'OnlineBackup': {'No': {'DeviceProtection': {'No': {'TechSupport': {'No': {'StreamingTV': {'No': {'StreamingMovies': {'No': {'TotalCharges': {0.0: 1,
                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [ ]:
def predict(inst,tree):
  for nodes in tree.keys():
    try:      
      value = inst[nodes]
      tree = tree[nodes][value]
    except KeyError:
      print("KeyError caught")
      return "KeyError"
    prediction = 0
        
    if type(tree) is dict:
        prediction = predict(inst, tree)
    else:
        prediction = tree
        break;                            
    
  return prediction

In [ ]:
y_pred = []
for i in range(len(test)):
  inst = test.iloc[i]
  #print(inst)
  prediction = predict(inst, tree)
  y_pred.append(prediction)


In [ ]:
y_pred = np.array(y_pred)
y_test = test['Target']
#print(y_test)
y_test = np.array(y_test)
print(y_test.shape, y_pred.shape)

from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))
print(metrics.confusion_matrix(y_test,y_pred))
print(metrics.accuracy_score(y_test,y_pred))

(1409,) (1409,)
              precision    recall  f1-score   support

           0       0.83      0.80      0.81      1027
           1       0.50      0.55      0.53       382

    accuracy                           0.73      1409
   macro avg       0.67      0.67      0.67      1409
weighted avg       0.74      0.73      0.73      1409

[[820 207]
 [172 210]]
0.7310149041873669


In [ ]:
key_error_list = []
y_pred = []
for i in range(len(train)):
  #print(i)
  inst = train.iloc[i]
  #print(inst)
  prediction = predict(inst, tree)
  if prediction == "KeyError":
    key_error_list.append(i)
    continue
  y_pred.append(prediction)

y_pred = np.array(y_pred)
y_train = train['Target']
y_train = np.array(y_train)

print(y_pred.shape)
print(y_train.shape)

print(metrics.classification_report(y_train, y_pred))
print(metrics.confusion_matrix(y_train,y_pred))
print(metrics.accuracy_score(y_train,y_pred))



(5634,)
(5634,)
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4147
           1       0.97      0.97      0.97      1487

    accuracy                           0.99      5634
   macro avg       0.98      0.98      0.98      5634
weighted avg       0.99      0.99      0.99      5634

[[4107   40]
 [  43 1444]]
0.9852680156194533


In [ ]:
##Initialize weights:
def initialize_weights(df):
  df['Weight'] = 1/len(df)

#initialize_weights(train)
#train.head()

In [ ]:
def weak_learner (df):
  #
  all_columns = list(df)
  y_column = all_columns.pop(all_columns.index('Target'))
  x_columns = all_columns

  max_depth = 1
  tree = Decision_Tree_Learning(df, x_columns, -1, max_depth-1, True)
  #
  return tree

In [ ]:
def calculate_error(y_pred, y_actual, df):
  error = 0
  indexes = []
  for i in range(len(y_pred)):
    if y_pred[i] != y_actual[i]:
      inst = df.iloc[i]
      wt = inst['Weight']
      error += wt
      indexes.append(i)
  return error, indexes

In [ ]:
def AdaBoost (examples, K):
  stump_list = [] #This is the list of d trees with depth 1
  Z_list = [] #Z is the amount of say
  error_list = []
  for k in range(K):
    df2 = examples.copy()
    L_weak = weak_learner(df2)

    weighted_examples = examples.copy()
    initialize_weights(weighted_examples)
    #print(examples)
    #print(weighted_examples)

    ##predict using the weak learner:
    y_pred = []
    for i in range(len(examples)):
      inst = examples.iloc[i]
      #print(inst)
      prediction = predict(inst, L_weak)
      y_pred.append(prediction)
    y_pred = np.array(y_pred)
    #print(y_pred)
    y_actual = examples['Target']
    y_actual = np.array(y_actual)
    #print(y_actual)
    df_temp = weighted_examples.copy()
    #print(y_pred, y_actual)
    error, error_indexes = calculate_error(y_pred, y_actual, df_temp) #0 indexed keep in mind
    if error > 0.5:
      continue
    error_list.append(error)
    stump_list.append(L_weak)
    #print(error, error_indexes)

    amount_of_say = log(((1-error)/(error + eps)) + eps)
    Z_list.append(amount_of_say)

    #print(weighted_examples)
    for i in range(len(weighted_examples)):
      if i not in error_indexes:
        #print(i)
        #print(weighted_examples.at[i, 'Weight'])
        tmp = weighted_examples.at[i, 'Weight'] * (error/((1 - error) + eps)) 
        weighted_examples.at[i, 'Weight'] = tmp
    #print(weighted_examples)

    ##Normalizing the weight:
    total_weight = weighted_examples['Weight'].sum()
    #print(total_weight)
    weighted_examples['Weight'] = weighted_examples['Weight'] / (total_weight + eps)
    weighted_examples['Weight'] = weighted_examples['Weight'] + eps
    #print(weighted_examples)
    #print(weighted_examples['Weight'].sum())

    sampled_examples = weighted_examples.sample(frac = 1, replace = True, random_state=1111, weights='Weight')
    #print(sampled_examples)
    examples = sampled_examples.reset_index(drop=True)
    examples.drop('Weight', axis=1, inplace=True)
    #print(examples)
  
  print(stump_list)
  print(Z_list)
  print(error_list)

  return stump_list, Z_list


In [ ]:
stump_list, Z_list = AdaBoost(train, 20)
#print(train)

[{'Contract': {'Month-to-month': 0, 'One year': 0, 'Two year': 0}}, {'Contract': {'Month-to-month': 1, 'One year': 0, 'Two year': 0}}, {'InternetService': {'DSL': 0, 'Fiber optic': 1, 'No': 0}}, {'tenure': {0.0: 0, 1.0: 1, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0, 6.0: 0, 7.0: 0, 8.0: 0, 9.0: 0, 10.0: 0, 11.0: 0, 12.0: 0, 13.0: 0, 14.0: 0, 15.0: 0, 16.0: 0, 17.0: 0, 18.0: 0}}, {'MonthlyCharges': {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 1, 5.0: 1, 6.0: 1, 7.0: 1, 8.0: 0, 9.0: 0, 10.0: 0, 11.0: 0, 12.0: 1, 13.0: 0, 14.0: 0, 15.0: 1, 16.0: 1, 17.0: 1, 18.0: 1}}, {'TotalCharges': {0.0: 1, 1.0: 1, 2.0: 1, 3.0: 1, 4.0: 0, 5.0: 1, 6.0: 1, 7.0: 1, 8.0: 1, 9.0: 0, 10.0: 1, 11.0: 0, 12.0: 0, 13.0: 1, 14.0: 1, 15.0: 0, 16.0: 0, 17.0: 0, 18.0: 0, 19.0: 0}}, {'tenure': {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 1, 4.0: 1, 5.0: 1, 6.0: 0, 7.0: 1, 8.0: 0, 9.0: 0, 10.0: 0, 11.0: 0, 12.0: 0, 13.0: 0, 14.0: 0, 15.0: 0, 16.0: 0, 17.0: 0, 18.0: 0}}, {'MonthlyCharges': {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0, 6.0: 0, 7.0: 0

In [ ]:
def AdaBoostPredict(inst):
  results = {}
  target_values = list(df['Target'].unique())
  #print(target_values)

  for value in target_values:
    results[value] = 0

  #print(results)

  for i in range (len(stump_list)):
    prediction = predict(inst, stump_list[i])
    amount_of_say = Z_list[i]
    results[prediction] = results[prediction] + amount_of_say
  #print(results)

  max = 0
  result = ""
  for value in target_values:
    if results[value] > max:
      max = results[value]
      result = value
  #print(result)
  return result

In [ ]:

y_pred_ada = []
for i in range(len(test)):
  inst = test.iloc[i]
  prediction = AdaBoostPredict(inst)
  y_pred_ada.append(prediction)

#print(y_pred_ada)
y_test = list(test['Target'])
#print(y_test)
y_pred_ada = np.array(y_pred_ada)
y_test = np.array(y_test)

print(metrics.classification_report(y_test, y_pred_ada))
print(metrics.confusion_matrix(y_test,y_pred_ada))
print(metrics.accuracy_score(y_test,y_pred_ada))


              precision    recall  f1-score   support

           0       0.79      0.90      0.84      1027
           1       0.57      0.37      0.45       382

    accuracy                           0.75      1409
   macro avg       0.68      0.64      0.65      1409
weighted avg       0.73      0.75      0.74      1409

[[920 107]
 [239 143]]
0.7544357700496807


In [ ]:
y_pred_ada = []
for i in range(len(train)):
  inst = train.iloc[i]
  prediction = AdaBoostPredict(inst)
  y_pred_ada.append(prediction)

#print(y_pred_ada)
y_train = list(train['Target'])
#print(y_train)
y_pred_ada = np.array(y_pred_ada)
y_train = np.array(y_train)

print(metrics.classification_report(y_train, y_pred_ada))
print(metrics.confusion_matrix(y_train,y_pred_ada))
print(metrics.accuracy_score(y_train,y_pred_ada))


              precision    recall  f1-score   support

           0       0.81      0.90      0.86      4147
           1       0.61      0.43      0.50      1487

    accuracy                           0.78      5634
   macro avg       0.71      0.66      0.68      5634
weighted avg       0.76      0.78      0.76      5634

[[3747  400]
 [ 855  632]]
0.7772452964146255


In [ ]:
a = metrics.confusion_matrix(y_train,y_pred_ada)

In [ ]:
print(a[0][1])

0
